In [4]:
import numpy as np
import pandas as pd
import sklearn as sk
import numpy.random as npr

In [87]:
#så vi ska observera 20 olika kunder och sannolikheten att de går bankrupt
#baserat på deras kategori av inkomst - hög, medel låg
sample_size = 20
np.random.seed(200)

obs_state = {'default':0, 'solvent':1}
obsran = npr.randint(low=0,high=2,size=20)


inv_obs_map = dict((i,j) for j, i in obs_state.items())
obs_seq = [inv_obs_map[v] for v in list(obsran)]



pi = [0.6, 0.4] #sannolikheten för de olika statesen
states = ['solvent', 'default']
hidden_states = ['hög', 'medel', 'låg']
pi = [0.02,0.28,0.7]
state_space = pd.Series(pi, index=hidden_states, name='states')
dfa = pd.DataFrame(columns=hidden_states, index=hidden_states)
dfa.loc[hidden_states[0]] = [0.3,0.3,0.4]
dfa.loc[hidden_states[1]] = [0.1, 0.45, 0.45]
dfa.loc[hidden_states[2]] = [0.2, 0.3, 0.5]
dfa = dfa.values

observable_states=states
dfb= pd.DataFrame(columns=observable_states, index=hidden_states)
dfb.loc[hidden_states[0]] = [1,0]
dfb.loc[hidden_states[1]] = [0.8,0.2]
dfb.loc[hidden_states[2]] = [0.2,0.8]
dfb = dfb.values

def viterbi(pi, a, b, obs):
    
    nStates = np.shape(b)[0]
    T = np.shape(obsran)[0]
    
    # första pathen
    path = path = np.zeros(T,dtype=int)
    # delta = högsta sannolikheten av vilken path som helst som når högsta stadiet
    delta = np.zeros((nStates, T))
    # phi --> argmax av Time (t) för varje steg
    phi = np.zeros((nStates, T))
    
    # initial delta och phi
    delta[:, 0] = pi * b[:, obsran[0]]
    phi[:, 0] = 0

     
    #'forward walk extension'
    for t in range(1, T):
        for s in range(nStates):
            delta[s, t] = np.max(delta[:, t-1] * a[:, s]) * b[s, obsran[t]] 
            phi[s, t] = np.argmax(delta[:, t-1] * a[:, s])
            print('s={s} and t={t}: phi[{s}, {t}] = {phi}'.format(s=s, t=t, phi=phi[s, t]))
    
    # find optimal path
    print('-'*50)
    print('Start Backtrace\n')
    path[T-1] = np.argmax(delta[:, T-1])
    for t in range(T-2, -1, -1):
        path[t] = phi[path[t+1], [t+1]]
        print('path[{}] = {}'.format(t, path[t]))
        
    return path, delta, phi

In [88]:
path, delta, phi = viterbi(pi, dfa, dfb, obsran)
state_map = {0:'hög', 1:'medel', 2:'låg'}
state_path = [state_map[v] for v in path]
pd.DataFrame().assign(Observation=obs_seq).assign(Best_Path=state_path)

s=0 and t=1: phi[0, 1] = 2.0
s=1 and t=1: phi[1, 1] = 1.0
s=2 and t=1: phi[2, 1] = 1.0
s=0 and t=2: phi[0, 2] = 2.0
s=1 and t=2: phi[1, 2] = 2.0
s=2 and t=2: phi[2, 2] = 2.0
s=0 and t=3: phi[0, 3] = 0.0
s=1 and t=3: phi[1, 3] = 1.0
s=2 and t=3: phi[2, 3] = 1.0
s=0 and t=4: phi[0, 4] = 0.0
s=1 and t=4: phi[1, 4] = 1.0
s=2 and t=4: phi[2, 4] = 1.0
s=0 and t=5: phi[0, 5] = 0.0
s=1 and t=5: phi[1, 5] = 1.0
s=2 and t=5: phi[2, 5] = 1.0
s=0 and t=6: phi[0, 6] = 2.0
s=1 and t=6: phi[1, 6] = 2.0
s=2 and t=6: phi[2, 6] = 2.0
s=0 and t=7: phi[0, 7] = 0.0
s=1 and t=7: phi[1, 7] = 1.0
s=2 and t=7: phi[2, 7] = 1.0
s=0 and t=8: phi[0, 8] = 2.0
s=1 and t=8: phi[1, 8] = 2.0
s=2 and t=8: phi[2, 8] = 2.0
s=0 and t=9: phi[0, 9] = 2.0
s=1 and t=9: phi[1, 9] = 2.0
s=2 and t=9: phi[2, 9] = 2.0
s=0 and t=10: phi[0, 10] = 0.0
s=1 and t=10: phi[1, 10] = 1.0
s=2 and t=10: phi[2, 10] = 1.0
s=0 and t=11: phi[0, 11] = 2.0
s=1 and t=11: phi[1, 11] = 2.0
s=2 and t=11: phi[2, 11] = 2.0
s=0 and t=12: phi[0, 12] = 2.0


,Observation,Best_Path
0,default,medel
1,solvent,låg
2,default,medel
3,default,medel
4,default,medel
5,solvent,låg
6,default,medel
7,solvent,låg
8,solvent,låg
9,default,medel


In [74]:
obsran

array([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1])

In [ ]:
#anledningen till att den bästa pathen för solvent är medelinkomst och
#inte hög, är pga den låga sannolikheten för att någon har en hög inkomst (pi)
